## ChromaDB を使った RAG のデモ

### OpenAI API Key とLLM、埋め込みモデルの設定

In [2]:
from dotenv import load_dotenv, find_dotenv
# 環境変数 OPENAI_API_KEY に OpenAI API Key を設定（.env ファイルの OPENAI_API_KEY からロード）
_ = load_dotenv(find_dotenv())
embedding_model = "text-embedding-3-large" # 埋め込みモデル
model = "gpt-4o" # LLM
temperature = 0 # LLMの生成のランダムさ

### LLMと埋め込みモデルの準備

In [3]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLMの準備
llm = ChatOpenAI(
    model=model, # モデル
    temperature=temperature, # ランダムさ
)

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### 検索対象の ChromaDB コレクション名の設定
コレクションとは、ChromaDB におけるベクトルデータの保存単位で、RDB のテーブルのようなもの

In [4]:
#collection_name = "mycollection_100" # 検索対象のChromadb コレクション名（チャンクサイズ100文字で事前に作成済み）
collection_name = "mycollection_200" # 検索対象のChromadb コレクション名（チャンクサイズ200文字で事前に作成済み）
#collection_name = "mycollection_500" # 検索対象のChromadb コレクション名（チャンクサイズ500文字で事前に作成済み）
#collection_name = "mycollection_1000" # 検索対象のChromadb コレクション名（チャンクサイズ1000文字で事前に作成済み）

### VectorStoreの準備

In [5]:
import chromadb
from langchain_chroma import Chroma

# LangChain の Chromadb VectorStore のインスタンスを作成
vector_store = Chroma(
    client=chromadb.PersistentClient(path="./chroma_db"),
    collection_name=collection_name,
    embedding_function=embeddings,
    collection_metadata={"hnsw:space": "ip"} 
)

### Retrieverの準備

In [6]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.45, "k": 20}
)

### PromptTemplateの準備

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや思考経過は出力しないでください。context の中の source のファイル名には、page_content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください、判断結果は出力しません。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: {context}"),
        ("human", "question: {input}"),
    ]
)

### RAGチェーンの準備

In [8]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | (lambda x: print(f"\n=== Generated Prompt ===\n{x}\n========================\n\n") or x)  # プロンプトを表示
    | llm
)

### 質問の定義

In [9]:
#query="ハグリッドがホグワーツへの入学案内書を持ってきたのはいつですか？それはどのような日でしたか？"
#query="アメリカの初代大統領は誰ですか？"
#query="時空管理局のアースラの艦長は誰ですか？"
#query="フェイトが留学生として通った小学校は？"
#query="ユーノ・スクライアは、普段どのような動物の姿で過ごしていますか？"
#query="聖王のゆりかごとは何ですか、どのくらいの大きさですか？"
#query="スターライトブレイカーは誰のどのような魔法ですか？"
#query="高町なのはが小学生のときに通っていた学校は？"
query="フェイトが王子様キャラと言われるのはなぜ？"


### RAGの実行

In [10]:
response = rag_chain.invoke(query)
print(f"=== LLM response ======\n{response}\n========================\n")
print(f"=== 回答 ===============\n{response.content}\n========================\n")



=== Generated Prompt ===
messages=[SystemMessage(content='あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや思考経過は出力しないでください。context の中の source のファイル名には、page_content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください、判断結果は出力しません。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: [Document(id=\'10f2bdd7-1166-4308-83b1-bed36dce1007\', metadata={\'source\': \'data\\\\Wikipedia-フェイト・テスタロッサ.pdf\'}, page_content=\'他にも運動神経が良く接近戦が多いためかフェイト役の水樹奈々やファンなどから王子様キャラだと思われがちだが実際はレヴィにセクハラをされ、真っ赤になるなどなのはよりも女の子らしい部分があり、『The MOVIE 1st』の告知の時、キャロから「守ってあげたくなるお姫様タイプ」と言われている。\'), Document(id=\'3cba60dc-ef2d-4b1f-8938-0dd5d256ceaf\', metadata={\'source\': \'data\\\\Wikipedia-高町なのは.pdf\'}, page_content=\'「エースオブエース」、「誰もが認める無敵のエース」などと呼ばれており、管理局内のみならずミッドチルダでは雑誌に取り上げられるような有名人となっている[6]。公の場でははやてらに対して敬語を用い、フェイトに対しても「フェイト隊⻑」と呼ぶことがあるがプライベートでは「はやてちゃん」「フェイトちゃん」「クロノくん」と、10年前同様の呼称を用い続けている。\'), Document(i